In [1]:
# -*- coding: utf-8 -*-
import matplotlib.pyplot as plt
import matplotlib
import math
import datetime
import networkx as nx
import os

In [2]:
def TodaysDate():
        
    Today = datetime.date.today()
    TodaysDate = Today.strftime('%d%b%Y')
    
    return TodaysDate

In [3]:
def DNACodingSequence(DNASequence, QualityScoreSequence, StartSequence, StopSequence):
#utilises ONLY ONE StopSequence, returns ONLY ONE CodingSequence
    
    QualityScoreString = """!"#$%&'()*+,-./0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[\]^_`abcdefghijklmnopqrstuvwxyz{|}~"""
    ThresholdQualityScore = 29 # ThresholdQualityScore must be between 0 and 93
    ThresholdQualityString = QualityScoreString[ThresholdQualityScore:]
    
    MinLength = 24
    MaxLength = 240
            
    StartIndex = DNASequence.find(StartSequence) + len(StartSequence)
    StopIndex = DNASequence.rfind(StopSequence)
    CodingSequence =  DNASequence[StartIndex:StopIndex]
    if MinLength <= len(CodingSequence) and len(CodingSequence) <= MaxLength and len(CodingSequence)%3 == 0:
        for Character in QualityScoreSequence[StartIndex:StopIndex]:
            if Character not in ThresholdQualityString:
                return None
        return str(CodingSequence)

In [4]:
def Translation(CodingSequence):
#translates DNA sequence

    TranslationCode = {
                    'AAA':'K','AAC':'N','AAG':'K','AAU':'N',
                    'ACA':'T','ACC':'T','ACG':'T','ACU':'T',
                    'AGA':'R','AGC':'S','AGG':'R','AGU':'S',
                    'AUA':'I','AUC':'I','AUG':'M','AUU':'I',
                    
                    'CAA':'Q','CAC':'H','CAG':'Q','CAU':'H',
                    'CCA':'P','CCC':'P','CCG':'P','CCU':'P',
                    'CGA':'R','CGC':'R','CGG':'R','CGU':'R',
                    'CUA':'L','CUC':'L','CUG':'L','CUU':'L',
                    
                    'GAA':'E','GAC':'D','GAG':'E','GAU':'D',
                    'GCA':'A','GCC':'A','GCG':'A','GCU':'A',
                    'GGA':'G','GGC':'G','GGG':'G','GGU':'G',
                    'GUA':'V','GUC':'V','GUG':'V','GUU':'V',
                    
                    'UAA':'#','UAC':'Y','UAG':'*','UAU':'Y',
                    'UCA':'S','UCC':'S','UCG':'S','UCU':'S',
                    'UGA':'&','UGC':'C','UGG':'W','UGU':'C',
                    'UUA':'L','UUC':'F','UUG':'L','UUU':'F'
                        }
    # UAA (ochre) — #
    # UAG (amber) — *
    # UGA (opal) — &
                    
    TranscriptionCode = {'A':'A','C':'C','G':'G','T':'U','U':'T'}
      
    RNASequence = ''
    for Nucleotide in CodingSequence:
        RNASequence += TranscriptionCode.get(Nucleotide,'X')
    #converts DNA to RNA
    #print RNASequence
        
    Peptide = ''
    while len(RNASequence) != 0:
        Peptide += TranslationCode.get(RNASequence[0:3],'Do not fuck with me!')
        RNASequence = RNASequence[3:]
    return Peptide

In [5]:
def SingleSelectionRoundSummary(fastqFileLocation):
#returns a list of lists with peptide-sequences and their frequencies, sorted by frequency in descending order
    
    RawDataFile = open(fastqFileLocation, 'r')
    Lines = RawDataFile.readlines()
    RawDataFile.close
    
    #StartSequence = 'ATG' # Met codon
    #StopSequence = 'TGCGGCAGC'# Akane seams to have trimmed siquences
    #StopSequence = 'TAG' # amber stop codon
    
    StartSequence = 'TAATACGACTCACTATAGGGTTAACTTTAAGAAGGAGATATACATATG'    # NNK - T7g10M.F48 
    StopSequence = 'TGCGGCAGCGGCAGCGGCAGCTAGGACGGGGGGCGGAAA' #NNK - CGS3an13.R39 
    #StartSequence = 'TAATACGACTCACTATAGGGTTGAACTTTAAGTAGGAGATATATCCATG'   #NNU - T7-CH-F49
    #StopSequence = 'TGTGGGTCTGGGTCTGGGTCTTAGGACGGGGGGCGGAAA'  #NNU - CGS3-CH-R39
    
    SingleSelectionRoundSummary = {}
    #creates empty SingleSelectionRoundSummary dictionary to store the results from a single round of selection
    #SingleSelectionRoundSummary = {PeptideY:    {CodingSequence_YZ:    Occurrence_YZ}}
        
    #populates SingleSelectionRoundSummary dictionary with the results from a single round of selection
    for i,Line in enumerate(Lines):
        if StartSequence in Line and StopSequence in Line:
            CodingSequence = DNACodingSequence(Line, Lines[i + 2], StartSequence, StopSequence)
            if CodingSequence != None:
                PeptideSequence = Translation(CodingSequence)
                if PeptideSequence not in SingleSelectionRoundSummary:
                    SingleSelectionRoundSummary[str(PeptideSequence)] = {str(CodingSequence) : 1}
                else:
                    if CodingSequence not in SingleSelectionRoundSummary[str(PeptideSequence)]:
                        SingleSelectionRoundSummary[str(PeptideSequence)][str(CodingSequence)] = 1
                    else:
                        SingleSelectionRoundSummary[str(PeptideSequence)][str(CodingSequence)] += 1

    return SingleSelectionRoundSummary

In [6]:
def HammingDistance(Sequence1, Sequence2):
    
    if len(Sequence1) < len(Sequence2):
        Sequence1 = Sequence1 + (len(Sequence2) - len(Sequence1)) * '%'
    elif len(Sequence1) > len(Sequence2):
        Sequence2 = Sequence2 + (len(Sequence1) - len(Sequence2)) * '%'
    
    HammingDistance = 0
    for i in range(len(Sequence1)):
        if Sequence1[i] == Sequence2[i]:
            HammingDistance = HammingDistance
        else:
            HammingDistance = HammingDistance + 1
            
    return HammingDistance

In [7]:
def HammingDistanceBasedFormating(Sequence1, Sequence2):
    
    if len(Sequence1) < len(Sequence2):
        Sequence1 = Sequence1 + (len(Sequence2) - len(Sequence1)) * '.'
    elif len(Sequence1) > len(Sequence2):
        Sequence2 = Sequence2 + (len(Sequence1) - len(Sequence2)) * '.'
    
    HammingDistance = 0
    FormatedSequence2 = ''
    for i in range(len(Sequence1)):
        if Sequence1[i] == Sequence2[i]:
            FormatedSequence2 += Sequence2[i].lower()
            HammingDistance = HammingDistance
        else:
            FormatedSequence2 += Sequence2[i]
            HammingDistance = HammingDistance + 1            
    return FormatedSequence2

In [8]:
def CompleteSelectionSummary(fastqDataFolderLocation):
# returns a SelectionSummary dictionary with the following structure
# SelectionSummary = {SelectionRound_X:    {PeptideXY:    {CodingDNA_XYZ:    Occurrence_XYZ}}}

    CompleteSelectionSummary = {}
    # creates empty SelectionSummary dictionary to store the results from all the rounds of selection

    for file in os.listdir(fastqDataFolderLocation):
        
        FileLocation = os.path.join(fastqDataFolderLocation, file)
          
        if file.endswith('.fastq'): # this conditional is necessary; without it some shit appears in the beginning of the file list
            RoundNumberFirstDigit = file[file.find('.')-2]
            RoundNumberSecondDigit = file[file.find('.')-1]
            if RoundNumberFirstDigit == '0':
                RoundNumber = int(RoundNumberSecondDigit)
                #print RoundNumber
            elif RoundNumberFirstDigit != '0':
                RoundNumber = int(file[file.find('.')-2 : file.find('.')])
                #print RoundNumber
        #(1.A) extracts the round number from the file name (file name should have two digit number before full stop — '00.') 
                
            SelectionRoundSummary = SingleSelectionRoundSummary(FileLocation)
            #(1.B) extracts single round results 
                    
            CompleteSelectionSummary[RoundNumber] = SelectionRoundSummary
            #(1.C) populate ConcatenatedResultsList
            #print ConcatenatedResultsList
            
    return CompleteSelectionSummary

In [9]:
def PeptidesOccurrences_BY_Round(fastqDataFolderLocation):
    SelectionSummary = CompleteSelectionSummary(fastqDataFolderLocation)
    
    PeptidesOccurrences_BY_Round = {}
    for Round in SelectionSummary:
        PeptidesOccurrences_IN_Round = {}
        for Peptide in SelectionSummary[Round]:
            PeptidesOccurrences_IN_Round[Peptide] = sum(SelectionSummary[Round][Peptide].values())
        PeptidesOccurrences_BY_Round[Round] = PeptidesOccurrences_IN_Round
        
    return PeptidesOccurrences_BY_Round

In [10]:
def DNAsOccurrences_BY_Round(fastqDataFolderLocation):
    SelectionSummary = CompleteSelectionSummary(fastqDataFolderLocation)
    
    DNAsOccurrences_BY_Round = {}
    for Round in SelectionSummary:
        DNAsOccurrences_IN_Round = {}
        for Peptide in SelectionSummary[Round]:
            for DNA in SelectionSummary[Round][Peptide]:
                DNAsOccurrences_IN_Round[DNA] = SelectionSummary[Round][Peptide][DNA]
        DNAsOccurrences_BY_Round[Round] = DNAsOccurrences_IN_Round

    return DNAsOccurrences_BY_Round

In [11]:
def TotalReads_BY_Round(fastqDataFolderLocation):
    SelectionSummary = CompleteSelectionSummary(fastqDataFolderLocation)
    Peptides_BY_Round = PeptidesOccurrences_BY_Round(fastqDataFolderLocation)
    
    TotalReads_BY_Round = {}
    for Round in SelectionSummary:
        TotalReads_BY_Round[Round] = sum(Peptides_BY_Round[Round].values())
        
    return TotalReads_BY_Round

In [12]:
def BaseRoundSortedPeptidesList(fastqDataFolderLocation, BaseRoundIndex):
    Peptides_BY_Round = PeptidesOccurrences_BY_Round(fastqDataFolderLocation)  
            
    PeptidesOccurrencesInBaseRound = Peptides_BY_Round[BaseRoundIndex]
    BaseRoundSortedPeptidesList = sorted(PeptidesOccurrencesInBaseRound, key = PeptidesOccurrencesInBaseRound.get, reverse = True)
    
    return BaseRoundSortedPeptidesList

In [13]:
def BaseRoundSortedDNAsList(fastqDataFolderLocation, BaseRoundIndex):
    DNAs_BY_Round = DNAsOccurrences_BY_Round(fastqDataFolderLocation)  
            
    DNAsOccurrences_IN_BaseRound = DNAs_BY_Round[BaseRoundIndex]
    BaseRoundSortedDNAsList = sorted(DNAsOccurrences_IN_BaseRound, key = DNAsOccurrences_IN_BaseRound.get, reverse = True)
    
    return BaseRoundSortedDNAsList

In [14]:
def DNAClonesOccurrences_BY_Round_BY_Peptide(fastqDataFolderLocation):
    SelectionSummary = CompleteSelectionSummary(fastqDataFolderLocation)
    
    DNAClonesOccurrences_BY_Round_BY_Peptide = {}
    for Round in SelectionSummary:
        DNAClonesOccurrences_BY_Peptide = {}
        for Peptide in SelectionSummary[Round]:
            DNAClonesOccurrences_BY_Peptide[Peptide] = len(SelectionSummary[Round][Peptide])
        DNAClonesOccurrences_BY_Round_BY_Peptide[Round] = DNAClonesOccurrences_BY_Peptide
        
    return DNAClonesOccurrences_BY_Round_BY_Peptide

In [15]:
def PeptidesAppearances_BY_Round(BaseRoundSortedPeptidesList, PeptidesOccurrences_BY_Round):
    
    PeptidesAppearances_BY_Round = {}
    
    for Peptide in BaseRoundSortedPeptidesList:
        PeptidesAppearances_BY_Round[Peptide] = []
        for Round in PeptidesOccurrences_BY_Round:
            if Peptide in PeptidesOccurrences_BY_Round[Round]:
                PeptidesAppearances_BY_Round[Peptide] += [Round]
    return PeptidesAppearances_BY_Round

In [16]:
def DNAsAppearances_BY_Round(BaseRoundSortedDNAsList, DNAsOccurrences_BY_Round):
    
    DNAsAppearances_BY_Round = {}
    
    for DNA in BaseRoundSortedDNAsList:
        DNAsAppearances_BY_Round[DNA] = []
        for Round in DNAsOccurrences_BY_Round:
            if DNA in DNAsOccurrences_BY_Round[Round]:
                DNAsAppearances_BY_Round[DNA] += [Round]
    return DNAsAppearances_BY_Round

In [17]:
def ComplementarySequence(String):
    DNAComplement = {'A':'T','C':'G','G':'C','T':'A'}
    RNAComplement = {'A':'U','C':'G','G':'C','U':'A'}
    
    ComplementarySequence = ''
    
    if 'T' in String:
        for i in range(len(String)):
            ComplementarySequence = ComplementarySequence + DNAComplement[String[i]]
    elif 'U' in String:
        for i in range(len(String)):
            ComplementarySequence = ComplementarySequence + RNAComplement[String[i]]
    return ComplementarySequence

In [18]:
def ReverseSequence(String):
    ReverseSequence = ''
    i = len(String) - 1
    while i >= 0:
        ReverseSequence = ReverseSequence + String[i]
        i -= 1
    return ReverseSequence

In [19]:
def SelectionSummaryReport(fastqDataFolderLocation, BaseRoundIndex, NumberOfTopPeptides, SelectionSummaryReportFileName):
    
    today = TodaysDate() 
    
    SelectionSummaryFileNameCSV = str(today) + 'SelectionSummary' + SelectionSummaryReportFileName + '.csv'
    SelectionSummaryReportFile = open(SelectionSummaryFileNameCSV, 'w')
    
    SelectionSummary = CompleteSelectionSummary(fastqDataFolderLocation)
    SortedRoundsList = sorted(SelectionSummary.keys())
    
    Peptides_BY_Round = PeptidesOccurrences_BY_Round(fastqDataFolderLocation)
    DNAs_BY_Round = DNAsOccurrences_BY_Round(fastqDataFolderLocation)
    TotalPeptides_BY_Round = TotalReads_BY_Round(fastqDataFolderLocation)

    BaseRoundSortedPeptides = BaseRoundSortedPeptidesList(fastqDataFolderLocation, BaseRoundIndex)
    BaseRoundTopSortedPeptides = BaseRoundSortedPeptides[0 : (NumberOfTopPeptides)]
    
    #print ('total reads by round ' + str(TotalPeptides_BY_Round))
    #print (len(DNAs_BY_Round[BaseRoundIndex]))
    #print (DNAs_BY_Round[BaseRoundIndex].keys())
    BaseRoundcDNAs = list(DNAs_BY_Round[BaseRoundIndex].keys())
    
    NNKCodons = ['AAG', 'CCT', 'CCG', 'GAT',
                 'TGT', 'TTT', 'GAG', 'GCT',
                 'GCG', 'ACG', 'ACT', 'TAT',
                 'CTG', 'TTG', 'CTT', 'TAG',
                 'TGG', 'ATT', 'CGG', 'CGT',
                 'AGG', 'ATG', 'GGT', 'GGG',
                 'CAG', 'GTT', 'GTG', 'TCT',
                 'TCG', 'AGT', 'AAT', 'CAT']
    
    CodonsNumber = 0
    NonNNKCodonsNumber = 0
    
    NonNNKDictionary = {}
    
    for cDNA in BaseRoundcDNAs:
        OriginalcDNA = cDNA
        NonNNKScore = 0
        while len(cDNA) != 0:
            CodonsNumber += 1
            if cDNA[0:3] not in NNKCodons:
                NonNNKCodonsNumber += 1
                NonNNKScore += 1
            cDNA = cDNA[3:]
        NonNNKDictionary[OriginalcDNA] = NonNNKScore
                              
    #print (CodonsNumber)
    #print (NonNNKCodonsNumber)
    #print (NonNNKDictionary)
    
    NNKcDNAs = []
    for cDNA in NonNNKDictionary:
        if NonNNKDictionary[cDNA] == 0:
            NNKcDNAs += [cDNA]
    
    print (NNKcDNAs)
    print (len(NNKcDNAs))
    print ('GTTTGGGATCCGAGGACGTTTTATTTGTCTAGGATT' in NNKcDNAs)
    
    
    
    
    BaseRoundTopPeptidecDNAs = SelectionSummary[BaseRoundIndex][BaseRoundTopSortedPeptides[0]]
    print (len(BaseRoundTopPeptidecDNAs))    
    
    TopPeptideClonesNonNNKDictionary = {}
    
    for cDNA in BaseRoundTopPeptidecDNAs:
        OriginalcDNA = cDNA
        NonNNKScore = 0
        while len(cDNA) != 0:
            CodonsNumber += 1
            if cDNA[0:3] not in NNKCodons:
                NonNNKCodonsNumber += 1
                NonNNKScore += 1
            cDNA = cDNA[3:]
        TopPeptideClonesNonNNKDictionary[OriginalcDNA] = NonNNKScore
            
                    
    #print (CodonsNumber)
    #print (NonNNKCodonsNumber)
    #print (NonNNKDictionary)
    
    TopPeptideClonesNNKcDNAs = []
    for cDNA in TopPeptideClonesNonNNKDictionary:
        if TopPeptideClonesNonNNKDictionary[cDNA] == 0:
            TopPeptideClonesNNKcDNAs += [cDNA]
    
    print (TopPeptideClonesNNKcDNAs)
    print (len(TopPeptideClonesNNKcDNAs))
    print ('GTTTGGGATCCGAGGACGTTTTATTTGTCTAGGATT' in TopPeptideClonesNNKcDNAs)

In [20]:
DataFolderLocation = '/Users/nikita/Documents/DataSequensing/PHD2/SequencingResults/PHD2 NNK'
BaseSelectionRoundNumber = 6
TopNPeptidesNumber = 24
SummaryFileName = 'Paper05_PHD2SelectionResults'

SelectionSummaryReport(DataFolderLocation, BaseSelectionRoundNumber, TopNPeptidesNumber, SummaryFileName)

82
['GTTTGGGATTCGAGGACGTTTTATTTGTCTAGGATT', 'GATAAGACTGGTGTGCTGTAGGAT', 'TGGGATCCTGCTACTATGTTTATTGCGCGTCGTTGT', 'GTTTGGGATCCTAGGACGTTTTATTTGTCTAGGATT', 'TGGGATCCTGCTACTATGTTTATTGTGCGTCGTTGT', 'TGGGATACTTCGACGGTTTATGTGCGGAGGCAG', 'GTTTGGGATACGAGGACGTTTTATTTGTCTAGGGTT', 'CGGACTTGGCCGTTGTATATTCCGATGTAT', 'TGGAATCCTCGGACTATTTTTATTAAGCGTGCT', 'GTTTGGGATCCGAGGACGTTTTATATGTCTAGGATT', 'GTTTGGGATCCGAGGACGTTTTATTTGTCTAGGAAT', 'TGGATTCATACTCATTATATTCATCGTACT', 'CAGCGTGATGTTGCGCCTTAGATTCGTGGTAATATG', 'ATTTGGGATACTGCTACTTTTTATCTGTCTAGGACT', 'GATGTTGTGAAGTTTGCTGTTATTGTGTGGTGTTAT', 'TCGTGGAATACTCGTTCGTTTTATCTTTCGCGGATT', 'TGGAATAGTTTTATTCTGTCTCGGGCTACTCATAAG', 'TGGTGGAATACTCGTTCGTTTTATTTGTCTAGGATT', 'GTTTGGGATCCGAGGACGTCTTATTTGTCTAGGATT', 'GTTCGTTTTCCGTTTTATATTCAGCGTAGG', 'GTTTGGGATCCGAGGACGTTTTATTTGTCTAGGTTT', 'TGGGATCCTGCTACTCTGTTTATTGTGCGTCGTTGT', 'GTTTGGGATCCGTGGACGTTTTATTTGTCTAGGATT', 'GTTCGGGATCCGAGGACGTTTTATTTGTCTAGGATT', 'ATTTGGGATACTGGTACTTTTCATCTGTCTAGGACT', 'GTTTGGGATCCGAGGACGTTTTATTAGTCTA

In [21]:
print(len('AATCCTAGACAAATAAAACGTCCTCGGATCCCAAAC'))

36


In [22]:
Translation = {'AAA':'K','AAC':'N','AAG':'K','AAU':'N',
                'ACA':'T','ACC':'T','ACG':'T','ACU':'T',
                'AGA':'R','AGC':'S','AGG':'R','AGU':'S',
                'AUA':'I','AUC':'I','AUG':'M','AUU':'I',
                
                'CAA':'Q','CAC':'H','CAG':'Q','CAU':'H',
                'CCA':'P','CCC':'P','CCG':'P','CCU':'P',
                'CGA':'R','CGC':'R','CGG':'R','CGU':'R',
                'CUA':'L','CUC':'L','CUG':'L','CUU':'L',
                
                'GAA':'E','GAC':'D','GAG':'E','GAU':'D',
                'GCA':'A','GCC':'A','GCG':'A','GCU':'A',
                'GGA':'G','GGC':'G','GGG':'G','GGU':'G',
                'GUA':'V','GUC':'V','GUG':'V','GUU':'V',
                
                'UAA':'ochre','UAC':'Y','UAG':'amber','UAU':'Y',
                'UCA':'S','UCC':'S','UCG':'S','UCU':'S',
                'UGA':'opal','UGC':'C','UGG':'W','UGU':'C',
                'UUA':'L','UUC':'F','UUG':'L','UUU':'F'}

ReverseTranslation = {}

for i in Translation:
    if Translation[i] not in ReverseTranslation:
        ReverseTranslation[Translation[i]] = [i]
    else:
        ReverseTranslation[Translation[i]] += [i]
        

print (ReverseTranslation)
print (ReverseTranslation['A'])
SortedReverseTranslation = sorted(ReverseTranslation)

NNKReverseTranslation = {}

for AminoAcid in SortedReverseTranslation:
    for Codon in ReverseTranslation[AminoAcid]:
        if Codon[2] == 'G' or Codon[2] == 'U':
            if AminoAcid not in NNKReverseTranslation:
                NNKReverseTranslation[AminoAcid] = [Codon]
            else:
                NNKReverseTranslation[AminoAcid] += [Codon]

print (NNKReverseTranslation)

NNKCodons = []
for AminoAcid in NNKReverseTranslation:
    NNKCodons += NNKReverseTranslation[AminoAcid]

print (NNKCodons)

{'I': ['AUA', 'AUU', 'AUC'], 'F': ['UUU', 'UUC'], 'H': ['CAU', 'CAC'], 'K': ['AAA', 'AAG'], 'M': ['AUG'], 'opal': ['UGA'], 'V': ['GUC', 'GUG', 'GUA', 'GUU'], 'Y': ['UAC', 'UAU'], 'Q': ['CAA', 'CAG'], 'P': ['CCG', 'CCC', 'CCU', 'CCA'], 'A': ['GCU', 'GCG', 'GCA', 'GCC'], 'W': ['UGG'], 'ochre': ['UAA'], 'S': ['UCA', 'UCG', 'UCU', 'AGC', 'UCC', 'AGU'], 'E': ['GAG', 'GAA'], 'D': ['GAU', 'GAC'], 'C': ['UGU', 'UGC'], 'T': ['ACA', 'ACC', 'ACG', 'ACU'], 'amber': ['UAG'], 'R': ['AGA', 'CGU', 'AGG', 'CGG', 'CGA', 'CGC'], 'G': ['GGU', 'GGA', 'GGG', 'GGC'], 'N': ['AAC', 'AAU'], 'L': ['UUA', 'UUG', 'CUG', 'CUC', 'CUU', 'CUA']}
['GCU', 'GCG', 'GCA', 'GCC']
{'Y': ['UAU'], 'I': ['AUU'], 'F': ['UUU'], 'amber': ['UAG'], 'P': ['CCG', 'CCU'], 'A': ['GCU', 'GCG'], 'H': ['CAU'], 'V': ['GUG', 'GUU'], 'K': ['AAG'], 'S': ['UCG', 'UCU', 'AGU'], 'E': ['GAG'], 'D': ['GAU'], 'Q': ['CAG'], 'C': ['UGU'], 'T': ['ACG', 'ACU'], 'W': ['UGG'], 'M': ['AUG'], 'G': ['GGU', 'GGG'], 'N': ['AAU'], 'R': ['CGU', 'AGG', 'CGG'], 'L